## Connect to Google Drive to access Dataset

In [ ]:
from google.colab import drive
import zipfile
import os

# Mount Google Drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/MyDrive

## Create Training And Testing Dataset

In [ ]:
train_data_dir= "/content/drive/MyDrive/Xray_train_data"

In [ ]:
test_data_dir= "/content/drive/MyDrive/Xray_train_data"

In [ ]:
IMAGE_SIZE = (256, 256)
IMAGE_SHAPE = IMAGE_SIZE + (3,)

##Importing The Libraries

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from glob import glob
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
img_height, img_width= IMAGE_SIZE
batch_size=16
train_datagen = ImageDataGenerator(rescale=1./255,zoom_range=0.2,shear_range=0.2,validation_split=0.3)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    subset='training')
validation_generator = train_datagen.flow_from_directory(
    train_data_dir, # same directory as training data
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    subset='validation')

In [ ]:
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.layers import Dense,Flatten
from tensorflow.keras.models import Model

In [ ]:
Xception = Xception(include_top=False,input_shape=(256,256,3))

In [ ]:
x = Flatten()(Xception.output)

In [ ]:
output = Dense(1, activation='sigmoid')(x)

In [ ]:
Xception = Model(Xception.input,output)

In [ ]:
Xception.summary()

In [ ]:
Xception.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
Xception.fit(train_generator,validation_data=validation_generator,epochs=5)

In [ ]:
Xception.save("Xception.h5")

In [ ]:
# Get predictions
probabilities = Xception.predict(validation_generator)

# Display the probabilities
print(probabilities)

In [ ]:
y_true = validation_generator.classes
y_true

In [ ]:
y_pred = probabilities > 0.5
y_pred

In [ ]:
from sklearn.metrics import confusion_matrix
cf_matrix = confusion_matrix(y_true, y_pred)
print(cf_matrix)

In [ ]:
import seaborn as sns
sns.heatmap(cf_matrix/np.sum(cf_matrix), annot=True,
            fmt='.2%', cmap='Blues')

In [ ]:
import numpy as np

def calculate_metrics(confusion_matrix):
    # Extract values from the confusion matrix
    TP = confusion_matrix[1, 1]
    TN = confusion_matrix[0, 0]
    FP = confusion_matrix[0, 1]
    FN = confusion_matrix[1, 0]

    # Calculate accuracy
    accuracy = (TP + TN) / np.sum(confusion_matrix)

    # Calculate precision
    precision = TP / (TP + FP) if (TP + FP) != 0 else 0

    # Calculate recall (sensitivity/TPR)
    recall = TP / (TP + FN) if (TP + FN) != 0 else 0

    # Calculate specificity
    specificity = TN / (TN + FP) if (TN + FP) != 0 else 0

    return accuracy, precision, recall, specificity

accuracy, precision, recall, specificity = calculate_metrics(cf_matrix)

print(f'Accuracy: {accuracy:.2f}')
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'Specificity: {specificity:.2f}')
